In [1]:
import os
import re
from pathlib import Path
from shutil import move

def get_label_filename(img_filename):
    base = os.path.splitext(img_filename)[0]
    if base.startswith(('cars_', 'pano_')):
        return base + '.txt'
    elif re.match(r'^\d', base):
        return 'cars_' + base + '.txt'
    else:
        return base + '.txt'

img_exts = {'.jpg', '.jpeg', '.png'}

# 처리할 하위 세트: train, val
for subset in ['train', 'test', 'val']:
    img_dir = Path(f'dataset/images/{subset}')
    lbl_dir = Path(f'dataset/labels/{subset}')

    for img_file in os.listdir(img_dir):
        src_img_path = img_dir / img_file
        if src_img_path.suffix.lower() not in img_exts:
            continue  # skip non-image files

        new_img_name = get_label_filename(img_file).replace('.txt', src_img_path.suffix)
        new_lbl_name = get_label_filename(img_file)

        if img_file != new_img_name:
            print(f"[INFO] ({subset}) 이미지 이름 변경: {img_file} -> {new_img_name}")
            move(str(src_img_path), str(img_dir / new_img_name))

            old_lbl_name = os.path.splitext(img_file)[0] + '.txt'
            old_lbl_path = lbl_dir / old_lbl_name
            new_lbl_path = lbl_dir / new_lbl_name

            if old_lbl_path.exists():
                print(f"[INFO] ({subset}) 라벨 이름 변경: {old_lbl_name} -> {new_lbl_name}")
                move(str(old_lbl_path), str(new_lbl_path))
            else:
                print(f"[WARNING] ({subset}) 라벨 파일 없음: {old_lbl_name}")

In [2]:
import os

base_path = 'dataset'
splits = ['train', 'val', 'test']

summary = {}

for split in splits:
    img_dir = os.path.join(base_path, 'images', split)
    label_dir = os.path.join(base_path, 'labels', split)
    
    images = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    labels = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])
    
    missing_labels = []
    empty_labels = []

    for img in images:
        label_name = os.path.splitext(img)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        
        if not os.path.exists(label_path):
            missing_labels.append(img)
        else:
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if not content:
                    empty_labels.append(img)
    
    summary[split] = {
        'image_count': len(images),
        'label_count': len(labels),
        'missing_labels': len(missing_labels),
        'empty_labels': len(empty_labels),
        'missing_label_files': missing_labels,
        'empty_label_files': empty_labels,
    }

for split, stats in summary.items():
    print(f"\n[{split.upper()} SET]")
    print(f"- 총 이미지 수: {stats['image_count']}")
    print(f"- 라벨 수: {stats['label_count']}")
    print(f"- 라벨 누락 이미지 수: {stats['missing_labels']}")
    print(f"- 빈 라벨 이미지 수: {stats['empty_labels']}")



[TRAIN SET]
- 총 이미지 수: 8020
- 라벨 수: 8020
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 0

[VAL SET]
- 총 이미지 수: 1154
- 라벨 수: 1154
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 0

[TEST SET]
- 총 이미지 수: 2308
- 라벨 수: 2308
- 라벨 누락 이미지 수: 0
- 빈 라벨 이미지 수: 0


In [3]:
import os

base_path = 'dataset'
splits = ['train', 'val', 'test']
deleted_files = []
skipped_files = []

for split in splits:
    img_dir = os.path.join(base_path, 'images', split)
    label_dir = os.path.join(base_path, 'labels', split)

    for label_file in os.listdir(label_dir):
        if not label_file.endswith('.txt'):
            continue

        label_path = os.path.join(label_dir, label_file)
        try:
            with open(label_path, 'r') as f:
                content = f.read().strip()
            if not content:  # 빈 라벨
                basename = os.path.splitext(label_file)[0]
                deleted_img = False
                for ext in ['.jpg', '.png', '.jpeg']:
                    img_path = os.path.join(img_dir, basename + ext)
                    if os.path.exists(img_path):
                        try:
                            os.remove(img_path)
                            deleted_files.append(img_path)
                            deleted_img = True
                        except Exception as e:
                            skipped_files.append((img_path, str(e)))
                        break

                try:
                    os.remove(label_path)
                    deleted_files.append(label_path)
                except Exception as e:
                    skipped_files.append((label_path, str(e)))
        except Exception as e:
            skipped_files.append((label_path, str(e)))

print(f"\n✅ 빈 라벨 이미지 및 라벨 총 {len(deleted_files)//2}개 세트 삭제 완료")
if skipped_files:
    print(f"⚠️ 삭제에 실패한 파일 {len(skipped_files)}개:")
    for f, reason in skipped_files:
        print(f"  - {f}: {reason}")



✅ 빈 라벨 이미지 및 라벨 총 5067개 세트 삭제 완료
